In [1]:
#Siamese Architecture_Simple_LSTM
#keras_1.1.2_theano_backend
#Dropout=0.2
#train:test:validation=6:2:2
#Using Wordvectors for words

In [2]:
import os
with open(os.path.expanduser('~')+'/.keras/keras.json','w') as f:
    new_settings = """{\r\n
    "epsilon": 1e-07,\r\n
    "image_data_format": "channels_last",\n
    "backend": "theano",\r\n
    "floatx": "float32"\r\n
    }"""
    f.write(new_settings)

import keras

Using Theano backend.


In [3]:
import pandas as pd
import nltk
import numpy as np
import gensim

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializations
from keras.callbacks import ModelCheckpoint

from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [5]:
#WordVectors
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [6]:
train_DF=pd.read_csv('train.csv',index_col='id')
train_DF.reset_index(drop=True,inplace=True)
train_DF['question1'] = train_DF['question1'].astype(str)
train_DF['question2'] = train_DF['question2'].astype(str)

In [7]:
#parameters for the Model
MAX_SEQUENCE_LENGTH = 250
MAX_NB_WORDS = 40000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

In [8]:
import re
numbers=re.compile(r'(\d+\.\d+|\d+)')
def cleanText(text):
    return numbers.sub('N',text)

In [9]:
texts = []
questions1=[]
questions2=[]
duplicates=[]

for idx in range(train_DF.question1.shape[0]):
    text=train_DF.question1[idx]
    text=cleanText(text)
    texts.append(text)
    questions1.append(text)
    
    text=train_DF.question2[idx]
    text=cleanText(text)
    texts.append(text)
    questions2.append(text)
    
    duplicates.append(train_DF.is_duplicate[idx])

In [10]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 90278 unique tokens.


In [11]:
questions1_sequences = tokenizer.texts_to_sequences(questions1)
questions2_sequences = tokenizer.texts_to_sequences(questions2)

In [12]:
questions1_sequences = pad_sequences(questions1_sequences, maxlen=MAX_SEQUENCE_LENGTH)
questions2_sequences = pad_sequences(questions2_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [13]:
questions2_sequences.shape

(404290, 250)

In [14]:
duplicates_1=duplicates
duplicates = to_categorical(np.asarray(duplicates))
print('Shape of label tensor:', duplicates.shape)

Shape of label tensor: (404290, 2)


In [15]:
VALIDATION_SPLIT=0.2
TEST_SPLIT=0.2

In [16]:
indices = np.arange(questions1_sequences.shape[0])
np.random.shuffle(indices)

questions1_sequences = questions1_sequences[indices]
questions2_sequences = questions2_sequences[indices]
duplicates = duplicates[indices]

nb_test_samples = int(TEST_SPLIT * questions1_sequences.shape[0])
nb_validation_samples = int((VALIDATION_SPLIT+TEST_SPLIT) * questions1_sequences.shape[0])

questions1_sequences_train = questions1_sequences[:-nb_validation_samples]
questions2_sequences_train = questions2_sequences[:-nb_validation_samples]
duplicates_train = duplicates[:-nb_validation_samples]

questions1_sequences_val = questions1_sequences[-nb_validation_samples:-nb_test_samples]
questions2_sequences_val = questions2_sequences[-nb_validation_samples:-nb_test_samples]
duplicates_val = duplicates[-nb_validation_samples:-nb_test_samples]

questions1_sequences_test = questions1_sequences[-nb_test_samples:]
questions2_sequences_test = questions2_sequences[-nb_test_samples:]
duplicates_test = duplicates[-nb_test_samples:]

In [17]:
print('Traing and validation set number of positive and negative reviews')
print(duplicates_train.sum(axis=0))
print(duplicates_val.sum(axis=0))
print(duplicates_test.sum(axis=0))

Traing and validation set number of positive and negative reviews
[ 153052.   89522.]
[ 50959.  29899.]
[ 51016.  29842.]


In [18]:
n_symbols = len(word_index) + 1 # adding 1 to account for 0th index (for masking)
embedding_weights = np.random.random((n_symbols,EMBEDDING_DIM))
for word,index in word_index.items():
    if word in model.wv.vocab:
        embedding_weights[index] = model.word_vec(word)       

In [19]:
embedding_weights.shape

(90279, 300)

In [20]:
embedding_layer=Embedding(input_dim=len(word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights=[embedding_weights],
                          input_length=MAX_SEQUENCE_LENGTH,
                          trainable=True)

In [25]:
#Input for each Question
left_question_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
right_question_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

#Encoders at Question Level
embedded_sequences = embedding_layer(left_question_input)
left_lstm_sent = LSTM(300,dropout_U=0.2,dropout_W=0.2)(embedded_sequences)

embedded_sequences = embedding_layer(right_question_input)
right_lstm_sent = LSTM(300,dropout_U=0.2,dropout_W=0.2)(embedded_sequences)

#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([left_lstm_sent,right_lstm_sent], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(2,activation='sigmoid')(both)

siamese_net = Model(input=[left_question_input,right_question_input],output=prediction)
siamese_net.load_weights("simple-lstm-dropout-2-word2vec-weights-improvement-00-0.449.hdf5")
siamese_net.compile(loss="binary_crossentropy",optimizer=Adam())
print("model - Siamese Networks using Simple LSTM")
print(siamese_net.summary())

model - Siamese Networks using Simple LSTM
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 250)           0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 250)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 250, 300)      27083700    input_3[0][0]                    
                                                                   input_4[0][0]                    
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 300)    

In [ ]:
#data_question2_1000_train.shape
#duplicates_1000_train=np.array(duplicates_1000_train).reshape(1000,1)
#duplicates_1000_vali=np.array(duplicates_1000_vali).reshape(1000,1)

# training

In [ ]:
# checkpoint
filepath="simple-lstm-dropout-2-word2vec-weights-improvement-{epoch:02d}-{val_loss:.3f}.hdf5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint1]
# Fit the model
siamese_net.fit([questions1_sequences_train,questions2_sequences_train], duplicates_train,
                validation_data=([questions1_sequences_val,questions2_sequences_val],duplicates_val)
                                  ,nb_epoch=3, batch_size=50,callbacks=callbacks_list,verbose=0)

# testing

In [27]:
scores = siamese_net.evaluate([questions1_sequences_test,questions2_sequences_test], duplicates_test, verbose=0)
print(scores)


0.506634959291


In [28]:
scores_train = siamese_net.evaluate([questions1_sequences_train,questions2_sequences_train], duplicates_train, verbose=0)
print(scores_train)


0.506607642972


In [29]:
scores_vali = siamese_net.evaluate([questions1_sequences_val,questions2_sequences_val], duplicates_val, verbose=0)
print(scores_vali)


0.507337769942
